In [277]:
from bs4 import BeautifulSoup as bs
import requests

In [278]:
base_url = 'https://sofifa.com'

year = 2018

if year == 2018:
    year_extension = '?lg%5B0%5D=13&v=18&e=158865&set=true' # BPL Fifa 18, Sept 28 2017
    date = '2018-09-28 00:00:00'
elif year == 2019:
    year_extension = '?lg%5B0%5D=13&v=19&e=159229&set=true'  # BPL Fifa 19, Sept 27 2018
    date = '2019-07-28 00:00:00'

teams_url = base_url + '/teams' + year_extension

column_headers = ['id', 'team_fifa_api_id', 'date', 'buildUpPlaySpeed', 'buildUpPlaySpeedClass',
                 'BuildUpPlayDribbling', 'BuildUpPlayDribblingClass', 'BuildupPlayPositioningClass',
                 'buildUpPlayPassing', 'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
                 'chanceCreationClass', 'chanceCreationCrossing', 'chanceCreationCrossingClass',
                 'chanceCreationShooting', 'chanceCreationShootingClass',
                 'chanceCreationPositioningClass', 'defencePressure', 'defencePressureClass',
                 'defenceAgression', 'defenceAgressionClass', 'defenceTeamWidth',
                 'defenceTeamWidthClass', 'defenceDefenderLineClass']

In [275]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup


def find_team_links(soup):
    table = soup.find('table', {'class': 'table table-hover persist-area'})  # Table of teams
    tbody = table.find('tbody')
    all_a = tbody.find_all('a', {'class': ''})
    return [base_url + link['href'] + year_extension for link in all_a if '/team/' in link['href']]


def get_team_attributes(soup):
    card_div = soup.find('div', {'class': 'card mb-2'})
    attributes_div = card_div.find('div', {'class': 'card-body'})
    data = attributes_div.find_all('dd')
    team_attributes = {}
    start = False
    i = 0
    for datum in data:
        category = datum.find('span', {'class': ['tooltip', 'multiline']})
        if start or 'Speed' in category.text:
            start = True
            if category is None:
                category = 'Dribbling'
            else:
                category = datum.find('span', {'class': ['tooltip', 'multiline']}).text
            data = datum.find('span', {'class': 'float-right'}).text.split()

            print(category, ':', data)
            team_attributes[category] = data
            i += 1
    print()
    return team_attributes


def get_all_team_attributes(team_urls):
    team_attributes = {}
    for team_url in team_urls:
        team_soup = soup_maker(team_url)
        team_attrs = get_team_attributes(team_soup)
        team_name = team_url.rsplit('/')[-2]
        team_attributes[team_name] = team_attrs
    return team_attributes

In [276]:
teams_soup = soup_maker(teams_url)
team_urls = find_team_links(teams_soup)
team_attributes = get_all_team_attributes(team_urls)

print(team_attributes)

Speed : ['30', 'Slow']
Dribbling : ['52', 'Normal']
Passing : ['30', 'Short']
Positioning : ['Free', 'Form']
Passing : ['34', 'Normal']
Crossing : ['48', 'Normal']
Shooting : ['58', 'Normal']
Positioning : ['Free', 'Form']
Pressure : ['60', 'Medium']
Aggression : ['78', 'Double']
Team Width : ['38', 'Normal']
Defender Line : ['Cover']

Speed : ['40', 'Balanced']
Dribbling : ['54', 'Normal']
Passing : ['37', 'Mixed']
Positioning : ['Organised']
Passing : ['35', 'Normal']
Crossing : ['35', 'Normal']
Shooting : ['35', 'Normal']
Positioning : ['Organised']
Pressure : ['67', 'High']
Aggression : ['65', 'Press']
Team Width : ['65', 'Normal']
Defender Line : ['Cover']

Speed : ['51', 'Balanced']
Dribbling : ['78', 'Lots']
Passing : ['50', 'Mixed']
Positioning : ['Organised']
Passing : ['65', 'Normal']
Crossing : ['58', 'Normal']
Shooting : ['62', 'Normal']
Positioning : ['Free', 'Form']
Pressure : ['66', 'Medium']
Aggression : ['68', 'Double']
Team Width : ['68', 'Wide']
Defender Line : ['Cov